# Medical Pilates AI Prescription System

이 노트북은 의료 필라테스 AI 처방 시스템의 주요 기능을 구현하고 테스트하는 환경을 제공합니다.

## 시스템 구성
1. 한국어 텍스트 처리 (Korean Text Processing)
2. RAG 시스템 (Retrieval-Augmented Generation)
3. 운동 데이터베이스 관리 (Exercise Database Management)
4. 웹 인터페이스 (Gradio Web Interface)

## 설정 방법
1. GitHub 저장소 클론
2. 필요한 패키지 설치
3. 환경 변수 설정
4. 모델 초기화

## 1. 초기 설정

### 1.1 GitHub 저장소 클론 및 패키지 설치

In [ ]:
!git clone https://github.com/ysl1016/medical-pilates-ai.git
%cd medical-pilates-ai
!pip install -r requirements.txt

### 1.2 환경 변수 설정
필요한 API 키와 환경 설정을 진행합니다.

In [ ]:
import os
from dotenv import load_dotenv

# API 키 설정
os.environ["OPENAI_API_KEY"] = "your-openai-api-key"  # 실제 키로 교체 필요
os.environ["APP_ENV"] = "development"

# 시스템 설정 로드
from config.settings import get_config
config = get_config()
config.initialize()

### 1.3 필요한 모듈 임포트

In [ ]:
from src.korean_utils import KoreanTextProcessor
from src.rag_utils import RAGSystem
from src.exercise_db import ExerciseDatabase, Exercise

# 클래스 초기화
korean_processor = KoreanTextProcessor()
exercise_db = ExerciseDatabase()
rag_system = RAGSystem()

## 2. 기본 기능 테스트

### 2.1 한국어 처리 테스트

In [ ]:
# 한국어 텍스트 처리 테스트
test_text = """
필라테스는 코어 강화와 자세 교정에 효과적인 운동입니다.
특히 허리 통증 환자들에게 추천되는 운동 방법입니다.
"""

# 번역 테스트
english_text = korean_processor.translate_ko_to_en(test_text)
print("영어 번역:", english_text)

# 키워드 추출 테스트
keywords = korean_processor.extract_keywords(test_text)
print("추출된 키워드:", keywords)

### 2.2 운동 데이터베이스 테스트

In [ ]:
# 운동 검색 테스트
beginner_exercises = exercise_db.search_exercises(difficulty="beginner")
print("초급 운동 목록:")
for exercise in beginner_exercises:
    print(f"- {exercise.name_ko} ({exercise.difficulty_level})")

# 운동 통계 확인
stats = exercise_db.get_exercise_statistics()
print("\n운동 데이터베이스 통계:")
print(json.dumps(stats, indent=2, ensure_ascii=False))

### 2.3 RAG 시스템 테스트

In [ ]:
# 질문-답변 테스트
query = "허리 통증이 있는 초보자에게 추천하는 필라테스 동작은 무엇인가요?"
response = rag_system.generate_response(query)
print("질문:", query)
print("\n답변:", response)

## 3. Gradio 웹 인터페이스

사용자 친화적인 웹 인터페이스를 구현합니다.

In [ ]:
import gradio as gr

def process_query(query: str, pain_level: int, experience_level: str):
    # 사용자 입력 처리
    context = {
        "pain_level": pain_level,
        "experience_level": experience_level
    }
    
    # RAG 시스템을 통한 응답 생성
    response = rag_system.generate_response(query, context)
    
    # 추천 운동 검색
    recommended_exercises = exercise_db.search_exercises(
        difficulty="beginner" if experience_level == "초보자" else "intermediate"
    )
    
    # 응답 포맷팅
    formatted_response = f"""
답변: {response}

추천 운동:
{''.join([f'- {ex.name_ko}\n' for ex in recommended_exercises[:3]])}

주의사항: 운동 시 통증이 발생하면 즉시 중단하고 전문가와 상담하세요.
"""
    
    return formatted_response

# Gradio 인터페이스 생성
iface = gr.Interface(
    fn=process_query,
    inputs=[
        gr.Textbox(label="질문을 입력하세요"),
        gr.Slider(minimum=0, maximum=10, step=1, label="통증 수준 (0-10)"),
        gr.Radio(["초보자", "중급자", "상급자"], label="운동 경험")
    ],
    outputs=gr.Textbox(label="AI 응답"),
    title="의료 필라테스 AI 상담 시스템",
    description="환자의 상태와 경험 수준에 맞는 필라테스 운동을 추천해드립니다."
)

# 인터페이스 실행
iface.launch(share=True)

## 마무리

이 노트북에서는 의료 필라테스 AI 시스템의 주요 기능을 테스트하고 웹 인터페이스를 구현했습니다.

### 주요 기능
1. 한국어 자연어 처리
2. 운동 데이터베이스 관리
3. RAG 기반 질의응답
4. 웹 기반 사용자 인터페이스

### 다음 단계
1. 더 많은 운동 데이터 추가
2. 모델 성능 개선
3. 사용자 피드백 수집 및 분석
4. 시스템 확장 및 개선